# Problem 13.1: Programming cellular response

<hr>

In [25]:
import numpy as np

import eqtk

<hr>

We have seen in this chapter that the combinatoric binding of multiple ligand types with multiple receptor types can lead to cellular addressing. In this problem, you will develop a scheme using two types of ligands, two types of "A" receptors, and two types of "B" receptors (as in this chapter) whereby the cell can exist respond to variation in ligand concentration in three orthogonal prescribed ways. We will following closely the work of [Su, et al., 2022](https://doi.org/10.1016/j.cels.2022.03.001).

We use the one-step promiscuous ligand-receptor model from this chapter, using the same notation. We define by a ligand "word" to be a set of specific concentration values for each ligand variant in a combination. For example, a concentration of 10 mM for ligand 1 and 100 mM for ligand 2 constitutes a ligand word (10 mM, 100 mM). In the diagram below, we have three ligand words.

<div style="width: 100px; margin: auto;">

![ligand_words](ligand_words.png)

</div>

Each word should encode a separate cellular response, giving different cell types. An example of three responses is shown in the image below.

<div style="width: 100px; margin: auto;">

![ligand_words](responses.png)

</div>

The cell "cares" about the levels in the colored regions; the response in the hatched regions is irrelevant. In the above diagram, purple indicates low response and yellow indicates a high response.

Your task is to come up with a set of parameters, eight $K_{ijk}$'s and eight $\epsilon_{ijk}$'s, along with three sets of receptor concentrations, each containing the concentrations of A₁, A₂, B₁, and B₂, that have the desired response above, one each for each of the three words in the top diagram. An example of a desired response is shown below.

<div style="width: 500px; margin: auto;">

![ligand_words](computed_responses.png)

</div>

In working this problem, it helps to have some handy functions from the chapter.

In [26]:
def make_rxns(nA, nB, nL):
    '''
    Generate trimolecular binding reactions for a system with
    nA, nB, and nL types of Type A receptors, Type B receptors,
    and ligands, respectively. Returns a single string.
    '''
    rxns = ""
    for k in range(nB):
        for j in range(nL):
            for i in range(nA):
                rxns += f"A_{i+1} + L_{j+1} + B_{k+1} <=> T_{i+1}_{j+1}_{k+1}\n"
    return rxns


def make_N(nA, nB, nL):
    """Generate a stoichiometric matrix for ligand-receptor binding with
    nA, nB, and nL types of Type A receptors, Type B receptors, and 
    ligands, respectively.
    """
    rxns = make_rxns(nA, nB, nL)
    N = eqtk.parse_rxns(rxns)

    # Sorted names
    names = sorted(N.columns, key=lambda s: (len(s), s))

    # Sorted columns
    N = N[names]

    # As a Numpy array
    return N.to_numpy(copy=True, dtype=float)


def readout(epsilon, c):
    """Readout function of a set of complex concentrations. This
    is the expression level of the genes under regulation of
    singaling from the receptors.
    """
    return np.dot(epsilon, c[:, -len(epsilon):].transpose())

We can make a stoichiometric matrix using these functions; it will be the same for all calculations we do.

In [28]:
N = make_N(2, 2, 2)

Half of the battle in problems like these is keeping track of indices and what species is what. The columns of the stoichiometric matrix correspond to the following species.

| index | species |
| ----------- | ----------- |
| 0 | A₁ |
| 1 | A₂ |
| 2 | B₁ |
| 3 | B₂ |
| 4 | L₁ |
| 5 | L₂ |
| 6 | A₁L₁B₁ |
| 7 | A₂L₁B₁ |
| 8 | A₁L₂B₁ |
| 9 | A₂L₂B₁ |
| 10 | A₁L₁B₂ |
| 11 | A₂L₁B₂ |
| 12 | A₁L₂B₂ |
| 13 | A₂L₂B₂ |

These indices also correspond to the species in our inputted initial concentrations and in the output of EQTK's solver for the equilibrium concentrations. With that in mind, we can write a function to generate our initial concentration values for a given set of initial ligand concentrations, `cL0`.

In [31]:
def make_c0_grid(cL0, cA10, cA20, cB10, cB20):
    """Create an array of initial concentrations.
    
    Parameters
    ----------
    cL0 : array_like
        Array of ligand concentrations. This is the same for ligand
        1 and ligand 2, so a single 1D array is all that is required.
    cA10 : float
        Total concentration of receptor A1.
    cA20 : float
        Total concentration of receptor A2.
    cB10 : float
        Total concentration of receptor A1.
    cB20 : float
        Total concentration of receptor B2.

    Returns
    -------
    output : 2D Numpy array
        If n is the length of the cL0 input array, the output
        is an n² by 14 array. Columns 0 through 3 are the total
        receptor concentrations. Columns 4 and 5 are the ligand
        concentrations. Columns 6 through 13 are the concentrations
        of all of the possible trimers (all set to zero).
    """
    # Number of ligand concentrations
    n = len(cL0)
    
    # Ligand concentrations
    cL0 = np.meshgrid(*tuple([cL0] * 2))

    # Initialize c0
    c0 = np.zeros((n**2, 14))

    # Add ligand concentrations
    c0[:, 4] = cL0[0].flatten()
    c0[:, 5] = cL0[1].flatten()
    
    # Add receptor concentrations
    c0[:, 0] = cA10
    c0[:, 1] = cA20
    c0[:, 2] = cB10
    c0[:, 3] = cB20

    return c0

In trying to find parameters, we will take

    cL0 = np.array([1.0, 32.0, 1000.0])

Next, in order to do the calculation, we need to specify the target response and for which ligand concentrations the target response is relevant. We refer to the latter as `active_target`, defined below.

In [37]:
# Ligand words from top diagram
active_target = np.array([0, 0, 1, 0, 0, 0, 1, 0, 1], dtype=bool)

You can see how it corresponds to the words by comparing to our target ligand concantrations.

In [38]:
c0 = make_c0_grid(np.array([1, 32, 1000]), 0, 0, 0, 0)
print("Ligand 1:     ", c0[:, 4])
print("Ligand 2:     ", c0[:, 5])
print("active target:", active_target)

Ligand 1:      [   1.   32. 1000.    1.   32. 1000.    1.   32. 1000.]
Ligand 2:      [   1.    1.    1.   32.   32.   32. 1000. 1000. 1000.]
active target: [False False  True False False False  True False  True]


The target is listed as active according to wherever we are considering a ligand word.

In [33]:
active_target.reshape((3, 3))

array([[False, False,  True],
       [False, False, False],
       [ True, False,  True]])

We can now specify a list of target responses.

In [39]:
targets = [
    np.array([0, 0, 0, 0, 0, 0, 1, 0, 0]), # Ligand 1 low, ligand 2 high
    np.array([0, 0, 1, 0, 0, 0, 0, 0, 0]), # Ligand 1 high, ligand 2 low
    np.array([0, 0, 0, 0, 0, 0, 0, 0, 1]), # Ligand 1 high, ligand 2 high
]

# Useful to know how many cell types we have
n_cell_types = len(targets)

Finally, it will be useful to have a function to solve for the normalized response of a cell to an inputted set of initial concentrations and parameters. The function returns the cellular response, with the maximum value of the response over all ligand concentrations being set to one.

**a)** Complete the function below.

In [ ]:
def solve_norm_response(c0, N, K, epsilon):
    """Solve for normalized response.
    """
    pass

**b)** To find optimal parameters, we will 

In [ ]:
def solve_readout(c0, N, K, epsilon):
    fixed_c = np.nan * np.ones_like(c0)
    fixed_c[:, 4:6] = c0[:, 4:6]
    c = eqtk.fixed_value_solve(c0=c0, fixed_c=fixed_c, N=N, K=K)
    c = eqtk.solve(c0=c0, N=N, K=K)
    return readout(epsilon, c)

Hint: The optimization will converge to a local minimum, which may not hit your target addressing very well. You can try rerunning the optimization with a different set of starting guesses for the parameters and receptor concentrations.

<br />